In [1]:
import numpy as np

In [2]:
def get_loss_probs(dice, sides):
    '''
    Calculate probability of defense losing each possible number of troops.
    
    Args:
        dice (tup): Number of dice for attack and defense, respectively
        sides (tup): Number of sides on dice for attack and defense, respectively
    
    Returns:
        dict (of floats): Probability defense suffers losses indicated in key
    '''

    # Enumerate all possible roll combinations. First rows are for attack
    # (up to 3) and last few for defense (up to 2).
    possible_sides = []
    for i in range(2):
        for _ in range(dice[i]):
            possible_sides.append(range(1,sides[i]+1))
    all_roll_combos = np.array(np.meshgrid(*possible_sides)).reshape(sum(dice),-1).T

    # Sort attack and defense columns separately.
    sort_rows_asc = lambda array2d: -np.array(map(sorted, -array2d))
    attack = sort_rows_asc(all_roll_combos[:,:dice[0]])
    defense = sort_rows_asc(all_roll_combos[:,dice[0]:])

    # Calculate number of troops defense loses in each engagement.
    total_losses = min(dice)
    defense_troops_lost = (attack[:,:total_losses] > defense[:,:total_losses]).sum(axis=1)

    # Calculate frequencies of each loss number for defense.
    freqs = {}
    for lost in defense_troops_lost:
        freqs[lost] = freqs.get(lost,0) + 1

    # Calculate probs by just dividing by number of combos.
    probs = {lost:freq/float(len(defense_troops_lost)) for lost,freq in freqs.iteritems()}

    return probs

In [3]:
def get_probs(sides):
    '''
    Args:
        sides (tup): Number of sides on dice for attack and defense, respectively
    
    Returns:
        dict (of dicts): Key is tuple for attack and defense dice. Value is
            output from get_loss_probs()
    '''
    probs = {}
    for atk in range(1,4):
        for dfn in range(1,3):
            dice = (atk, dfn)
            probs[dice] = get_loss_probs(dice, sides)
    return probs

In [4]:
get_probs((6,6))

{(1, 1): {0: 0.5833333333333334, 1: 0.4166666666666667},
 (1, 2): {0: 0.7453703703703703, 1: 0.25462962962962965},
 (2, 1): {0: 0.4212962962962963, 1: 0.5787037037037037},
 (2, 2): {0: 0.44830246913580246,
  1: 0.32407407407407407,
  2: 0.22762345679012347},
 (3, 1): {0: 0.3402777777777778, 1: 0.6597222222222222},
 (3, 2): {0: 0.2925668724279835,
  1: 0.3357767489711934,
  2: 0.37165637860082307}}